In [13]:
train.data <- read.csv(file.path("..", "data", "training_data.csv"))
test.data <- read.csv(file.path("..", "data", "test_data.csv"))

In [4]:
set.seed(100)
numeric.intensity <- as.numeric(train.data$Intensity)
x <- train.data[, -c(2,3)]
x$Intensity <- numeric.intensity
n.before <- dim(x)[2] #numbers of predictors before the reduction
idx.zero.var <- apply(x, 2, var) == 0
x <- x[,!idx.zero.var]
y <- train.data$VALENCE.PLEASANTNESS

## Boosting 1

In [5]:
#searching for input variables with zero variance (without the varible Intensity)
set.seed(100)
x <- train.data[, -c(1,2,3)]

idx.zero.var <- apply(x, 2, var) == 0
x <- x[,!idx.zero.var]


#attache Intensity as factor
x$Intensity <- as.factor(train.data$Intensity)

data <- x
data$VALENCE.PLEASANTNESS <- train.data$VALENCE.PLEASANTNESS


In [7]:
set.seed(100)
#train and validation indexes
len <- length(x[,1])
idx.train <- sample(1:len, 2*len/3)

#xgboost does not accept data frames therefore we will first convert the data into ordinary matrices
library(xgboost)
library(Matrix)
train.x <- sparse.model.matrix(VALENCE.PLEASANTNESS ~ . -1, data = data[idx.train,])
validation.x <- sparse.model.matrix(VALENCE.PLEASANTNESS ~ . -1, data = data[-idx.train,])
train.y <- data$VALENCE.PLEASANTNESS[idx.train]
validation.y <- data$VALENCE.PLEASANTNESS[-idx.train]

In [ ]:
boost.heart <- xgboost(train.x, label = train.y,
                      objective = "reg:squarederror",
                      eta = 0.01,
                      max_depth = 2,
                      nround = 500)

In [9]:
prediction.train <- predict(boost.heart, train.x)
prediction.validation <- predict(boost.heart, validation.x)
MSE.train <- mean((prediction.train - train.y)^2)
MSE.validation <- mean((prediction.validation - validation.y)^2)


In [10]:
MSE.train
MSE.validation

[1] 300.1458

[1] 439.2583

In [ ]:
library(xgboost)
library(Matrix)
#Boosting Submission
set.seed(100)
#Preparation of training and test data
train <- train.data[, -c(1,2,3)]
idx.zero.var <- apply(train, 2, var) == 0

train <- train[,!idx.zero.var]
test <- test.data[,-c(1,2)]
test <- test[,!idx.zero.var]


#test$Intensity <- as.factor(test.data$Intensity)
train$Intensity <- as.numeric(train.data$Intensity)-1
train
#test intensity is always at level high, so that the prediction function has a problem (cheat with adding a row that afterwards is substracted)
#test <- rbind(test, train[1,])
train.x = train

train$VALENCE.PLEASANTNESS <- train.data$VALENCE.PLEASANTNESS
train.y = train$VALENCE.PLEASANTNESS

train.x <- sparse.model.matrix(VALENCE.PLEASANTNESS ~ . -1, data = train)
#test.x <- sparse.model.matrix(VALENCE.PLEASANTNESS ~ . -1, data = test)
train.y <- train$VALENCE.PLEASANTNESS
#validation.y <- data$VALENCE.PLEASANTNESS[-idx.train]

In [82]:
test <- test.data[,-c(1,2)]
test <- test[,!idx.zero.var]
test$Intensity <- as.numeric(test.data$Intensity)-1

In [ ]:
boost.heart <- xgboost(train.x, label = train.y,
                      objective = "reg:squarederror",
                      eta = 0.01,
                      max_depth = 2,
                      nround = 500)

In [79]:
prediction.boost = predict(boost.heart, as.matrix(test))
submission <- data.frame(Id = 1:68, VALENCE.PLEASANTNESS = prediction.boost)
write.csv(submission, file = "../Submissions/boosting2.csv", row.names = FALSE)


## Boosting 2 - Regularized gradient boosting 

One difference between boosting and random forests: in boosting, because the growth of a particular tree takes into account the other trees that have already been grown, smaller trees are typically sufficient (less splits and depth)

In [3]:
library(xgboost)

In [1]:
set.seed(100)
len <- length(x[,1])
idx.train <- sample(1:len, 3*len/4)

train.x <- x[idx.train,]
train.y <- y[idx.train]
test.x <- x[-idx.train,]
test.y <- y[-idx.train]


ERROR: Error in eval(expr, envir, enclos): objet 'x' introuvable


In [5]:
dtrain = xgb.DMatrix(data =  as.matrix(train.x), label = train.y )
dtest = xgb.DMatrix(data =  as.matrix(test.x), label = test.y)

In [6]:
watchlist = list(train=dtrain, test=dtest)

In [52]:
bst = xgb.train(data = dtrain, 
                max.depth = 8, 
                eta = 0.3, 
                nthread = 2, 
                nround = 1000, 
                watchlist = watchlist, 
                objective = "reg:squarederror", 
                early_stopping_rounds = 50,
                print_every_n = 500)

[1]	train-rmse:36.493153	test-rmse:37.327053 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 50 rounds.

Stopping. Best iteration:
[7]	train-rmse:10.149774	test-rmse:24.547579



We get a train-rmse of 10.15 and a test-rmse of 24.55. But, the parameters were chosen randomly
Now, let's tune the algorithm with 3 parameters : 
1) The number of trees 

2) The shrinkage parameter lambda : Typical values are 0.01 or 0.001, and the right choice can depend on the problem. Very small λ can require using a very large value of B in order to achieve good performance.

3) The number of splits in each tree, which controls the complexity of the boosted ensemble (controlled with max.depth)

Let's run a slower learning model, by reducing the learning rate, and reducing the number of splits in each tree

In [53]:
bst_slow = xgb.train(data = dtrain, 
                        max.depth=5, 
                        eta = 0.01, 
                        nthread = 2, 
                        nround = 10000, 
                        watchlist = watchlist, 
                        objective = "reg:squarederror", 
                        early_stopping_rounds = 50,
                        print_every_n = 500)

[1]	train-rmse:48.982822	test-rmse:47.107918 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 50 rounds.

Stopping. Best iteration:
[280]	train-rmse:12.274206	test-rmse:23.773617



We can see that we reduced the test-rmse by 6%. The problem here : What we have done here is fit to the training set and the test set at the same time (leading to model overfit). 

 We need to work with a validation set and only at the end evaluate the model performance against the test set.

In [54]:
len <- length(train.x[,1])
idx.valid <- sample(1:len, 2*len/3)

train.val.x <- train.x[idx.valid,]
train.val.y <- train.y[idx.valid]

valid.x <- train.x[-idx.valid,]
valid.y <- train.y[-idx.valid]


In [55]:
gb_train = xgb.DMatrix(data = as.matrix(train.val.x), label = train.val.y )
gb_valid = xgb.DMatrix(data = as.matrix(valid.x), label = valid.y )


In [56]:
watchlist = list(train = gb_train, valid = gb_valid)

In [57]:
bst_slow = xgb.train(data= gb_train, 
                        max.depth = 10, 
                        eta = 0.01, 
                        nthread = 2, 
                        nround = 10000, 
                        watchlist = watchlist, 
                        objective = "reg:squarederror", 
                        early_stopping_rounds = 50,
                        print_every_n = 500)

[1]	train-rmse:49.436710	valid-rmse:48.084187 
Multiple eval metrics are present. Will use valid_rmse for early stopping.
Will train until valid_rmse hasn't improved in 50 rounds.

Stopping. Best iteration:
[305]	train-rmse:5.971423	valid-rmse:24.474022



In [58]:
y_hat_valid = predict(bst_slow, dtest)
test_mse = mean(((y_hat_valid - test.y)^2))
test_mse
test_rmse = sqrt(test_mse)
test_rmse

[1] 671.6173

[1] 25.91558

This is higher then on the first run, but we can be confident that the improved score is not due to overfit thanks to our use of a validation set! A lower rmse isn't necessarily better if it comes at the cose of overfit, we now have more confidence in external predictions.



Let's find the best hyper-parameter combinations

In [59]:
max.depths = c(7, 9)
etas = c(0.1,0.01, 0.001)

best_params = 0
best_score = 0

count = 1

for( depth in max.depths ) {
    for(num in etas) {

        bst_grid = xgb.train(data = gb_train, 
                                max.depth = depth, 
                                eta=num, 
                                nthread = 2, 
                                nround = 10000, 
                                watchlist = watchlist, 
                                objective = "reg:squarederror", 
                                early_stopping_rounds = 50, 
                                verbose=0)

        if(count == 1){
            best_params = bst_grid$params
            best_score = bst_grid$best_score
            count = count + 1
            }
        else if( bst_grid$best_score < best_score){
            best_params = bst_grid$params
            best_score = bst_grid$best_score
        }
    }
}

best_params
best_score

$max_depth
[1] 7

$eta
[1] 0.001

$nthread
[1] 2

$objective
[1] "reg:squarederror"

$validate_parameters
[1] TRUE

[1] 23.79161

In [60]:
bst_tuned = xgb.train( data = gb_train, 
                        max.depth = 7, 
                        eta = 0.001, 
                        nthread = 2, 
                        nround = 10000, 
                        watchlist = watchlist, 
                        objective = "reg:squarederror", 
                        early_stopping_rounds = 50,
                        print_every_n = 500)

y_hat_xgb_grid = predict(bst_tuned, dtest)

test_mse = mean(((y_hat_xgb_grid - test.y)^2))
test_rmse = sqrt(test_mse)

[1]	train-rmse:49.839478	valid-rmse:48.421314 
Multiple eval metrics are present. Will use valid_rmse for early stopping.
Will train until valid_rmse hasn't improved in 50 rounds.

[501]	train-rmse:32.446823	valid-rmse:34.793262 
[1001]	train-rmse:21.792664	valid-rmse:27.858894 
[1501]	train-rmse:15.200837	valid-rmse:24.959738 
[2001]	train-rmse:11.097453	valid-rmse:23.966742 
Stopping. Best iteration:
[2400]	train-rmse:8.857841	valid-rmse:23.791611



In [61]:
test_rmse

[1] 25.70601

We should now maybe try to submit to Kaggle to see if we get better on the test set